In [1]:
dataset = 'lastfm'

domain = {
    'lastfm': 'music',
    'dbbook': 'book',
    'movielens': 'movie'
}[dataset]

items = {
    'lastfm': 'artist',
    'dbbook': 'book',
    'movielens': 'movie'
}[dataset]
import os
datapath = os.path.join('..', 'data', dataset)

special_token = '<|reserved_special_token_22|>'

resp_frac = 0.5

In [2]:
import pandas as pd
import os
import random
import numpy as np
import swifter  
user_item_df_train = pd.read_csv(os.path.join(datapath, 'user-item', 'train.tsv'), \
            names=['user', 'item', 'rating'], sep='\t')

user_item_df_test = pd.read_csv(os.path.join(datapath, 'user-item', 'test.tsv'), \
            names=['user', 'item', 'rating'], sep='\t')


df = pd.concat([user_item_df_train, user_item_df_test], ignore_index=True)
all_items = df['item'].unique()

In [3]:
if dataset=='movielens':
    def fix_title(title):
        if ", The (" in title:
            name_film, _, year = title.rpartition(", The (")
            title = "The " + name_film + " (" + year
            return title
        if ", A (" in title:
            name_film, _, year = title.rpartition(", A (")
            title = "A " + name_film + " (" + year
        return title
    df_movies = pd.read_csv(os.path.join(datapath, r"movies.dat"), sep="::", names=["item_id", "name", "geners"], encoding='ISO-8859-1')
    df_movies['name'] = df_movies['name'].apply(lambda x: fix_title(x)[0])
    #df_movies['year'] = df_movies['name'].apply(lambda x: fix_title(x)[1])
    import re

    df_relations = pd.read_csv(os.path.join(datapath, 'mapping_entities.tsv'), \
            names=['url', 'id_set'], sep='\t')
    dbpedia_mapping = pd.read_csv(os.path.join(datapath, 'MappingMovielens2DBpedia-1.2.tsv'), \
            names=['id_movie', 'name', 'dbpedia_url'], sep='\t')
    df_movies = dbpedia_mapping.set_index('dbpedia_url').join(df_relations.set_index('url')).reset_index()
    df_movies['name'] = df_movies['name'].apply(lambda x: re.sub("\s+\(\d+\)$", "", x))
    df_movies['name'] = df_movies['name'].apply(lambda x: re.sub(r"[^a-zA-Z0-9]+", ' ', x).strip())
    df_movies.dropna(inplace=True)
    df_movies['id_set'] = df_movies['id_set'].astype(int)
    mapping_dict = df_movies.loc[:, ['id_set', 'name']].set_index('id_set').to_dict()['name']
    property_mapping = pd.read_json(os.path.join(datapath, 'entities_names.json'), typ='series')
    mapping_relation = pd.read_csv(os.path.join(datapath, 'mapping_relations.tsv'), \
            names=['rel', 'id'], sep='\t')
    
if dataset=='dbbook':
    df_relations = pd.read_csv(os.path.join(datapath, 'mapping_entities.tsv'), sep='\t')
    df_relations['name'] = df_relations['uri'].apply(lambda x: x.split(";")[0])
    temp = df_relations[df_relations['id'].isin(df['item'].unique())]
    mapping_dict = temp.set_index('id').to_dict()['name']
    property_mapping = pd.read_json(os.path.join(datapath, 'entities_names.json'), typ='series')
    mapping_relation = pd.read_csv(os.path.join(datapath, 'mapping_relations.tsv'), \
            names=['id', 'rel'], sep='\t')
    
if dataset=='lastfm':
    df_relations = pd.read_csv(os.path.join(datapath, 'mapping_items.tsv'), \
            names=['id', 'name'], sep='\t')
    mapping_dict = df_relations.set_index('id').to_dict()['name']
    property_mapping = pd.read_json(os.path.join(datapath, 'entities_names.json'), typ='series')
    mapping_relation = pd.read_csv(os.path.join(datapath, 'mapping_relations.tsv'), \
            names=['rel', 'id'], sep='\t')

In [4]:
pd.to_pickle(df['item'].value_counts().to_dict(), os.path.join(datapath,"item_popularity.pkl"))

In [ ]:
df['item'].value_counts().to_dict()

# Test items

In [6]:
dataset = 'movielens'

domain = {
    'lastfm': 'music',
    'dbbook': 'book',
    'movielens': 'movie'
}[dataset]

items = {
    'lastfm': 'artist',
    'dbbook': 'book',
    'movielens': 'movie'
}[dataset]
import os
datapath = os.path.join('..', 'data', dataset)

special_token = '<|reserved_special_token_22|>'

resp_frac = 0.5

In [7]:
import pandas as pd
import numpy as np
import itertools

def _get_resp_ids(row):
    items = np.array(row['item'])
    return [int(x) for x in set(list(items[row['index_like_resp'] + row['index_dislike_resp']]))]

temp = pd.read_pickle(os.path.join(datapath, f"{dataset}_test_dataset.pkl"))

pd.to_pickle(list(set(itertools.chain(*temp.apply(_get_resp_ids, axis = 1).tolist()))), os.path.join(datapath, "test_items.pkl"))